In [1]:
import os
import warnings
import re
import random
import math
import pandas as pd
import numpy as np
import scipy
import sklearn

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [4]:
feeds_df = pd.read_csv(os.getcwd() + '/Datasets/feeds_df_for_reco.csv')
feeds_df['createdAt'] = pd.to_datetime(feeds_df['createdAt'])
feeds_df['updatedAt'] = pd.to_datetime(feeds_df['updatedAt'])
feeds_df = feeds_df[~feeds_df['text'].isna()]
print(feeds_df.shape)
feeds_df.head()

(1740, 13)


,contentId,anonymous,authorId,createdAt,isActive,isDelete,points,text,type,updatedAt,resource_link,resource_videoUrl,resource_image
0,5df457077e4ea600161b8e53,False,5df4561e7e4ea600161b8e3e,2019-12-14 03:29:11.865,True,False,0.007634,"Hello, my name is Kalyani. I have done my bach...",QUESTION,2020-02-06 05:14:28.877,[],[],[]
1,5df45c817e4ea600161b8f00,False,5df45a987e4ea600161b8ec4,2019-12-14 03:52:34.000,True,False,0.015267,My name is Vartika. I am interested in photogr...,QUESTION,2020-02-06 05:14:28.925,[],[],[]
2,5df456257e4ea600161b8e41,False,5df454d27e4ea600161b8e2a,2019-12-14 03:25:25.300,True,False,0.007634,Hello. I am Yashi. I have just started freelan...,POST,2020-02-06 05:14:28.872,[],[],[]
3,5df462ba7e4ea600161b9016,False,5df462387e4ea600161b8ffc,2019-12-14 04:19:06.651,True,True,0.007634,Hi! I’m Neha and I have just started out as a ...,QUESTION,2020-02-17 07:11:03.815,[],[],[]
4,5df46b807e4ea600161b9170,False,5df468337e4ea600161b9129,2019-12-14 04:56:32.858,True,False,0.015267,interested in learning the skills of a beautic...,QUESTION,2020-02-06 05:14:29.217,[],[],[]


# Interactions Data

In [5]:
likes_df = pd.read_csv(os.getcwd() + '/Datasets/likes_coll_df.csv')
likes_df['createdAt'] = pd.to_datetime(likes_df['createdAt'])
likes_df['updatedAt'] = pd.to_datetime(likes_df['updatedAt'])
likes_df = likes_df[likes_df['type'] != 'ARTICLE']
print(likes_df.shape)
likes_df.head()

(4456, 9)


,likeId,commentId,createdAt,emojiType,isActive,referenceId,type,updatedAt,userId
0,5df736726c5e063edee5d014,5df4b118a68eb700165a6665,2019-12-16 07:46:58.348,Like,True,NaN,COMMENT,2019-12-16 07:46:58.348,5df734e4d092c70016203ba9
1,5df737896c5e063edee5e632,NaN,2019-12-16 07:51:37.315,Like,True,5df736d3d092c70016203bca,QUESTION,2019-12-16 07:51:37.315,5df72831d092c70016203a8e
2,5df7494e6c5e063edee70dde,5df73f56d092c70016203c84,2019-12-16 09:07:26.908,Like,True,NaN,COMMENT,2019-12-16 09:07:26.908,5df4bb29a68eb700165a66bc
3,5df78b7a6c5e063edeeae69c,NaN,2019-12-16 13:49:46.842,Like,True,5df76b0a11788b0016813416,POST,2019-12-16 13:49:46.842,5df789f511788b0016813438
4,5df7bbfe6c5e063edeed8ea8,5df75af7d092c70016203db1,2019-12-16 17:16:46.086,Like,True,NaN,COMMENT,2019-12-16 17:16:46.086,5df7ba6e11788b0016813478


In [6]:
comments_df = pd.read_csv(os.getcwd() + '/Datasets/comments_df.csv')
comments_df['createdAt'] = pd.to_datetime(comments_df['createdAt'])
print(comments_df.shape)
comments_df.head()

(4426, 8)


,commentId,authorId,createdAt,isActive,isSubComment,referenceId,type,comment_text
0,5df479d37e4ea600161b9425,5df467277e4ea600161b9109,2019-12-14 05:57:39.052,True,False,5df45c817e4ea600161b8f00,QUESTION,"Vartika, I have been through this and it took ..."
1,5df475b27e4ea600161b934e,5df465bd7e4ea600161b90d0,2019-12-14 05:40:02.158,True,False,5df45ec37e4ea600161b8f68,POST,"Hi Swati. Definitely, blogging can really take..."
2,5df480917e4ea600161b94da,5df4579f7e4ea600161b8e68,2019-12-14 06:26:25.577,True,True,5df458dd7e4ea600161b8e8f,POST,Thanks Seema! This was very helpful!
3,5df48ae87e4ea600161b9626,5df46cf57e4ea600161b9191,2019-12-14 07:10:32.012,True,False,5df488727e4ea600161b95e8,QUESTION,Hi Mona. It is really important for us to make...
4,5df4a998a68eb700165a65cf,5df4651e7e4ea600161b9099,2019-12-14 09:21:28.821,True,False,5df465017e4ea600161b9091,QUESTION,"Hi. After the camera, laptop seems to me to be..."


In [7]:
bookmarks_df = pd.read_csv(os.getcwd() + '/Datasets/bookmarks_coll_df.csv')
bookmarks_df['createdAt'] = pd.to_datetime(bookmarks_df['createdAt'])
bookmarks_df['updatedAt'] = pd.to_datetime(bookmarks_df['updatedAt'])
bookmarks_df = bookmarks_df[bookmarks_df['type'] != 'COMPANY']
print(bookmarks_df.shape)
bookmarks_df.head()

(126, 7)


,bookmarkId,createdAt,isActive,referenceFeedId,type,updatedAt,userId
0,5e5e1df0038098664944f081,2020-03-03 09:05:52.348,True,5e5e195568860c00163d49db,POST,2020-03-16 09:37:54.993,5df4c077a68eb700165a66c3
2,5e60a430c268fb81a0fe786b,2020-03-05 07:03:12.363,True,5e5e195568860c00163d49db,POST,2020-03-05 07:04:25.611,5df72831d092c70016203a8e
3,5e60c1c4c268fb81a0002f7d,2020-03-05 09:09:24.755,True,5e60a54c83660f0016b76f9b,POST,2020-03-05 09:09:24.755,5df4c077a68eb700165a66c3
4,5e7f1dd4c268fb81a0cf1465,2020-03-28 09:50:11.912,True,5e1320d6ce1fa00016e8344d,POST,2020-03-28 09:50:11.912,5e7ee338c3f6090022544a33
5,5e809622c268fb81a0f79044,2020-03-29 12:35:46.432,True,5e80784d894de50024c4c674,QUESTION,2020-03-29 12:35:46.432,5e7b4c8a5e2628002444fd04


In [8]:
likes_df.columns

Index(['likeId', 'commentId', 'createdAt', 'emojiType', 'isActive',
       'referenceId', 'type', 'updatedAt', 'userId'],
      dtype='object')

In [9]:
comments_df.columns

Index(['commentId', 'authorId', 'createdAt', 'isActive', 'isSubComment',
       'referenceId', 'type', 'comment_text'],
      dtype='object')

In [10]:
bookmarks_df.columns

Index(['bookmarkId', 'createdAt', 'isActive', 'referenceFeedId', 'type',
       'updatedAt', 'userId'],
      dtype='object')

In [11]:
post_likes_df = likes_df[likes_df['type'] != 'COMMENT']
print(post_likes_df.shape)
post_likes_df = post_likes_df[['likeId', 'referenceId', 'userId', 'createdAt', 'type']]
post_likes_df = post_likes_df.rename(columns = {'likeId':'eventId', 'referenceId':'contentId', 'userId':'personId'})
post_likes_df['eventType'] = 'LIKE'

(2490, 9)


In [12]:
likes_df_formatted = post_likes_df
likes_df_formatted.head()

,eventId,contentId,personId,createdAt,type,eventType
1,5df737896c5e063edee5e632,5df736d3d092c70016203bca,5df72831d092c70016203a8e,2019-12-16 07:51:37.315,QUESTION,LIKE
3,5df78b7a6c5e063edeeae69c,5df76b0a11788b0016813416,5df789f511788b0016813438,2019-12-16 13:49:46.842,POST,LIKE
6,5df86ea96c5e063edef77c12,5df78b5d11788b0016813449,5df765c9d092c70016203dd3,2019-12-17 05:59:05.461,POST,LIKE
7,5df89d146c5e063edefa6340,5df8772411788b0016813595,5df45b5b7e4ea600161b8ede,2019-12-17 09:17:08.555,POST,LIKE
8,5df8bab86c5e063edefc3cef,5df78b5d11788b0016813449,5df50ced3a629f001633613c,2019-12-17 11:23:36.737,POST,LIKE


In [13]:
comments_df_formatted = comments_df[['commentId', 'referenceId', 'authorId', 'createdAt', 'type', 'isSubComment']]
comments_df_formatted = comments_df_formatted.rename(columns = {'commentId':'eventId', 'referenceId':'contentId', 'authorId':'personId'})
comments_df_formatted['eventType'] = np.where(comments_df_formatted['isSubComment'] == False, 'COMMENT', 'SUBCOMMENT')
comments_df_formatted = comments_df_formatted.drop(['isSubComment'], axis=1)
comments_df_formatted.head()


,eventId,contentId,personId,createdAt,type,eventType
0,5df479d37e4ea600161b9425,5df45c817e4ea600161b8f00,5df467277e4ea600161b9109,2019-12-14 05:57:39.052,QUESTION,COMMENT
1,5df475b27e4ea600161b934e,5df45ec37e4ea600161b8f68,5df465bd7e4ea600161b90d0,2019-12-14 05:40:02.158,POST,COMMENT
2,5df480917e4ea600161b94da,5df458dd7e4ea600161b8e8f,5df4579f7e4ea600161b8e68,2019-12-14 06:26:25.577,POST,SUBCOMMENT
3,5df48ae87e4ea600161b9626,5df488727e4ea600161b95e8,5df46cf57e4ea600161b9191,2019-12-14 07:10:32.012,QUESTION,COMMENT
4,5df4a998a68eb700165a65cf,5df465017e4ea600161b9091,5df4651e7e4ea600161b9099,2019-12-14 09:21:28.821,QUESTION,COMMENT


In [14]:
bookmarks_df_formatted = bookmarks_df[['bookmarkId', 'referenceFeedId', 'userId', 'createdAt', 'type']]
bookmarks_df_formatted = bookmarks_df_formatted.rename(columns = {'bookmarkId':'eventId', 'referenceFeedId':'contentId', 'userId':'personId'})
bookmarks_df_formatted['eventType'] = 'BOOKMARK'
bookmarks_df_formatted.head()

,eventId,contentId,personId,createdAt,type,eventType
0,5e5e1df0038098664944f081,5e5e195568860c00163d49db,5df4c077a68eb700165a66c3,2020-03-03 09:05:52.348,POST,BOOKMARK
2,5e60a430c268fb81a0fe786b,5e5e195568860c00163d49db,5df72831d092c70016203a8e,2020-03-05 07:03:12.363,POST,BOOKMARK
3,5e60c1c4c268fb81a0002f7d,5e60a54c83660f0016b76f9b,5df4c077a68eb700165a66c3,2020-03-05 09:09:24.755,POST,BOOKMARK
4,5e7f1dd4c268fb81a0cf1465,5e1320d6ce1fa00016e8344d,5e7ee338c3f6090022544a33,2020-03-28 09:50:11.912,POST,BOOKMARK
5,5e809622c268fb81a0f79044,5e80784d894de50024c4c674,5e7b4c8a5e2628002444fd04,2020-03-29 12:35:46.432,QUESTION,BOOKMARK


In [15]:
interactions_df = likes_df_formatted.append([comments_df_formatted, bookmarks_df_formatted])
print(interactions_df.shape)
interactions_df.head()

(7042, 6)


,eventId,contentId,personId,createdAt,type,eventType
1,5df737896c5e063edee5e632,5df736d3d092c70016203bca,5df72831d092c70016203a8e,2019-12-16 07:51:37.315,QUESTION,LIKE
3,5df78b7a6c5e063edeeae69c,5df76b0a11788b0016813416,5df789f511788b0016813438,2019-12-16 13:49:46.842,POST,LIKE
6,5df86ea96c5e063edef77c12,5df78b5d11788b0016813449,5df765c9d092c70016203dd3,2019-12-17 05:59:05.461,POST,LIKE
7,5df89d146c5e063edefa6340,5df8772411788b0016813595,5df45b5b7e4ea600161b8ede,2019-12-17 09:17:08.555,POST,LIKE
8,5df8bab86c5e063edefc3cef,5df78b5d11788b0016813449,5df50ced3a629f001633613c,2019-12-17 11:23:36.737,POST,LIKE


In [16]:
interaction_summary = interactions_df.groupby(['personId'])['contentId'].nunique().reset_index()
interaction_summary.columns = ['personId', 'content_ct']
interaction_summary = interaction_summary.sort_values(['content_ct'], ascending=False)
print("Total Users:", interaction_summary.shape)
print("Users with more than 1 content interaction:", interaction_summary[interaction_summary['content_ct']>1].shape)
interaction_summary.head()


Total Users: (1044, 2)
Users with more than 1 content interaction: (470, 2)


,personId,content_ct
97,5df4c077a68eb700165a66c3,164
100,5df72831d092c70016203a8e,155
226,5e783e20ea7e44002251594b,152
134,5df7877d11788b0016813430,136
113,5df734e4d092c70016203ba9,125


# Data Wrangling

In [17]:
event_type_strength = {
#   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
#   'FOLLOW': 3.0,
   'COMMENT': 4.0,
    'SUBCOMMENT': 4.0
}

interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])

In [24]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_1_interactions_df = users_interactions_count_df[users_interactions_count_df == 1].reset_index()[['personId']]
print('# users with 1 interactions: %d' % len(users_with_1_interactions_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 2].reset_index()[['personId']]
print('# users with at least 2 interactions: %d' % len(users_with_enough_interactions_df))

# users: 1044
# users with 1 interactions: 574
# users with at least 2 interactions: 470


In [25]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 2 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 7042
# of interactions from users with at least 2 interactions: 6252


In [ ]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# Evaluation

In [ ]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

In [ ]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [ ]:
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

In [ ]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    if(((pd.Series(person_id).isin(pd.Series(interactions_df.index.unique().to_list()))) == True)[0]):
        interacted_items = interactions_df.loc[person_id]['contentId']
        return(set(interacted_items if type(interacted_items) == pd.Series else [interacted_items]))
    else:
        interacted_items = set()
        return(interacted_items)
    

In [ ]:
def get_not_interacted_items_sample(person_id, sample_size, seed=42):
    interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
    all_items = set(feeds_df['contentId'])
    non_interacted_items = all_items - interacted_items

    random.seed(seed)
    non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
    return set(non_interacted_items_sample)

In [ ]:
def _verify_hit_top_n(item_id, recommended_items, topn):
    try:
        index = next(i for i, c in enumerate(recommended_items) if c == item_id)
    except:
        index = -1
    hit = int(index in range(0, topn))
    return hit, index

In [ ]:
def evaluate_model_for_user(model, person_id):
    #Getting the items in test set
    interacted_values_testset = interactions_test_indexed_df.loc[person_id]
    if type(interacted_values_testset['contentId']) == pd.Series:
        person_interacted_items_testset = set(interacted_values_testset['contentId'])
    else:
        person_interacted_items_testset = set([interacted_values_testset['contentId']])  
    interacted_items_count_testset = len(person_interacted_items_testset) 
    
    #Getting a ranked recommendation list from a model for a given user
    person_recs_df = model.recommend_items(person_id, 
                                         items_to_ignore=get_items_interacted(person_id, 
                                         interactions_train_indexed_df), 
                                         topn=10000000000)
    hits_at_5_count = 0
    hits_at_10_count = 0
    #For each item the user has interacted in test set
    for item_id in person_interacted_items_testset:
        #Getting a random sample (100) items the user has not interacted 
        #(to represent items that are assumed to be no relevant to the user)
        non_interacted_items_sample = get_not_interacted_items_sample(person_id, 
                                                      sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                      seed=int(re.sub(r'[a-zA-Z]', r'', item_id))%(2**32))

        #Combining the current interacted item with the 100 random items
        items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

        #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
        valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
        valid_recs = valid_recs_df['contentId'].values
        
        #Verifying if the current interacted item is among the Top-N recommended items
        hit_at_5, index_at_5 = _verify_hit_top_n(item_id, valid_recs, 5)
        hits_at_5_count += hit_at_5
        hit_at_10, index_at_10 = _verify_hit_top_n(item_id, valid_recs, 10)
        hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
    recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
    recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

    person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
    return person_metrics, person_recs_df[0:10]

In [ ]:
def evaluate_model(model):
    #print('Running evaluation for users')
    people_metrics = []
    reco_list = pd.DataFrame(columns = ['contentId', 'eventStrength'])
    
    for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
        #if idx % 100 == 0 and idx > 0:
        #    print('%d users processed' % idx)
        #print(person_id)
        person_metrics, rec_list = evaluate_model_for_user(model, person_id)  
        person_metrics['_person_id'] = person_id
        people_metrics.append(person_metrics)
        reco_list = reco_list.append(rec_list)
        
    print('%d users processed' % idx)

    detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
    global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
    global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
    global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}
    
    
    return global_metrics, detailed_results_df, reco_list

# Popularity Model

In [ ]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

In [ ]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, feeds_df)

In [ ]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df, full_reco_list = evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

In [ ]:
print("Coverage:", full_reco_list['contentId'].nunique() / feeds_df['contentId'].nunique())

# Content-Based Filtering model

In [ ]:
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
stopwords_list = stopwords.words('english')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = feeds_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(feeds_df['text'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

In [ ]:
def get_item_profile(item_id):
    #print(item_id)
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    #print(ids)
    if(isinstance(ids, str)):
        ids = [ids]
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    #print(person_id)
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
                                                   .isin(feeds_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        #print(person_id)
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [ ]:
user_profiles = build_users_profiles()
len(user_profiles)

In [ ]:
myprofile = user_profiles['5df4c077a68eb700165a66c3']
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles['5df4c077a68eb700165a66c3'].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(feeds_df)

In [ ]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df, full_reco_list = evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

In [ ]:
print("Coverage:", full_reco_list['contentId'].nunique() / feeds_df['contentId'].nunique())

# Collaborative Filtering model

### Matrix Factorization

In [ ]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix_df.head(10)

In [ ]:
users_items_pivot_matrix_df.shape

In [ ]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

In [ ]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

In [ ]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
users_items_pivot_sparse_matrix

In [ ]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)

In [ ]:
U.shape

In [ ]:
Vt.shape

In [ ]:
sigma = np.diag(sigma)
sigma.shape

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
print(all_user_predicted_ratings.shape)
all_user_predicted_ratings

In [ ]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [ ]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

In [ ]:
len(cf_preds_df.columns)

In [ ]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, feeds_df)

In [ ]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df, full_reco_list = evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

In [ ]:
print("Coverage:", full_reco_list['contentId'].nunique() / feeds_df['contentId'].nunique())

In [ ]:
cf_recommender_model.recommend_items(user_id = '5df4c077a68eb700165a66c3', items_to_ignore=[], topn=10, verbose=False)

### Hybrid Recommendation 

In [ ]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df, cb_ensemble_weight=1.0, cf_ensemble_weight=1.0):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.cb_ensemble_weight = cb_ensemble_weight
        self.cf_ensemble_weight = cf_ensemble_weight
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose,
                                                           topn=1000).rename(columns={'recStrength': 'recStrengthCB'})
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_rec_model.recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, 
                                                           topn=1000).rename(columns={'recStrength': 'recStrengthCF'})
        
        #Combining the results by contentId
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'outer', 
                                   left_on = 'contentId', 
                                   right_on = 'contentId').fillna(0.0)
        
        #Computing a hybrid recommendation score based on CF and CB scores
        #recs_df['recStrengthHybrid'] = recs_df['recStrengthCB'] * recs_df['recStrengthCF'] 
        recs_df['recStrengthHybrid'] = (recs_df['recStrengthCB'] * self.cb_ensemble_weight) \
                                     + (recs_df['recStrengthCF'] * self.cf_ensemble_weight)
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('recStrengthHybrid', ascending=False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrengthHybrid', 'contentId']]


        return recommendations_df
    
hybrid_recommender_model = HybridRecommender(content_based_recommender_model, cf_recommender_model, feeds_df,
                                             cb_ensemble_weight=1.0, cf_ensemble_weight=100.0)


In [ ]:
hybrid_recommender_model.recommend_items(user_id = '5df4c077a68eb700165a66c3', items_to_ignore=[], topn=10, verbose=False)

In [ ]:
print('Evaluating Hybrid model...')
hybrid_global_metrics, hybrid_detailed_results_df, full_reco_list = evaluate_model(hybrid_recommender_model)
print('\nGlobal metrics:\n%s' % hybrid_global_metrics)
hybrid_detailed_results_df.head(10)

In [ ]:
print("Coverage:", full_reco_list['contentId'].nunique() / feeds_df['contentId'].nunique())

# Comparing Methods

In [ ]:
global_metrics_df = pd.DataFrame([cb_global_metrics, pop_global_metrics, cf_global_metrics, hybrid_global_metrics]) \
                        .set_index('modelName')
global_metrics_df

In [ ]:
%matplotlib inline
ax = global_metrics_df.transpose().plot(kind='bar', figsize=(15,8))
for p in ax.patches:
    ax.annotate("%.3f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
popularity_model.recommend_items(user_id = 'dummy', items_to_ignore=[], topn=10, verbose=False)

In [ ]:
item_popularity_df.head(10)